                                 NHIỆM VỤ 1: ĐỌC VÀ LÀM SẠCH DỮ LIỆU


In [5]:
## 1. Đọc dữ liệu từ thư mục data_raw
##Dữ liệu gốc được giữ nguyên, không chỉnh sửa, chỉ đọc để xử lý.

import pandas as pd
import numpy as np

df_patient = pd.read_csv("../data_raw/patient_info.csv")
df_vaccine = pd.read_csv("../data_raw/vaccination_record.csv")
df_reaction = pd.read_csv("../data_raw/reaction_log.csv")

In [6]:
## 2. Khảo sát cấu trúc dữ liệu
##Kiểm tra số dòng, số cột, kiểu dữ liệu và các lỗi định dạng ban đầu.
from IPython.display import display
def tong_hop_thong_tin_bang(df, name):
    print(f"\n===== TỔNG HỢP BẢNG {name.upper()} =====")
    print(f"Số dòng: {df.shape[0]}")
    print(f"Số cột : {df.shape[1]}")
    print("Kiểu dữ liệu các cột:")
    print(df.dtypes)

# Hiển thị dữ liệu mẫu 
display(df_patient.head(3))
display(df_vaccine.head(3))
display(df_reaction.head(3))

# Tổng hợp cấu trúc
tong_hop_thong_tin_bang(df_patient, "patient_info")
tong_hop_thong_tin_bang(df_vaccine, "vaccination_record")
tong_hop_thong_tin_bang(df_reaction, "reaction_log")


,patient_id,age_group,gender
0,BN500,6-17,Female
1,bn501,0-5,m
2,bn502,50+,nam


,patient_id,vaccine_type,dose_number,vaccination_date
0,BN535,DPT,1st,2024-02-23
1,bn520,covid19,3,2024-03-07
2,BN528,DPT,2,23/03/2024


,patient_id,reaction_type,severity,report_date
0,bn536,đau tay,severe,2024-04-28
1,BN505,Đau tay,severe,2024-03-29
2,bn519,đau tay,2,16/04/2024



===== TỔNG HỢP BẢNG PATIENT_INFO =====
Số dòng: 40
Số cột : 3
Kiểu dữ liệu các cột:
patient_id    object
age_group     object
gender        object
dtype: object

===== TỔNG HỢP BẢNG VACCINATION_RECORD =====
Số dòng: 120
Số cột : 4
Kiểu dữ liệu các cột:
patient_id          object
vaccine_type        object
dose_number         object
vaccination_date    object
dtype: object

===== TỔNG HỢP BẢNG REACTION_LOG =====
Số dòng: 80
Số cột : 4
Kiểu dữ liệu các cột:
patient_id       object
reaction_type    object
severity         object
report_date      object
dtype: object


In [7]:
##3.Chuẩn hoá mã và nhãn quan trọng: patient_id, gender (M/F), age_group, vaccine_type.
##Loại bỏ khoảng trắng và chuẩn hóa viết hoa để đảm bảo khả năng merge giữa các bảng.

def clean_patient_id(df):
    df["patient_id"] = (
        df["patient_id"]
        .astype(str)
        .str.strip()
        .str.upper()
    )
    return df

df_patient  = clean_patient_id(df_patient)
df_vaccine  = clean_patient_id(df_vaccine)
df_reaction = clean_patient_id(df_reaction)


gender_map = {
    "nam": "M", "male": "M", "m": "M", "1": "M",
    "nữ": "F", "nu": "F", "female": "F", "f": "F", "0": "F"
}

df_patient["gender"] = (
    df_patient["gender"]
    .astype(str)
    .str.lower()
    .str.strip()
    .map(gender_map)
)

age_map = {
    "0_5": "0-5", "0–5": "0-5", "0-5": "0-5",
    "6_12": "6-12", "6-12": "6-12",
    "6_17": "6-17", "13_17": "13-17", "13-17": "13-17",
    "18_49": "18-49", "18-49": "18-49",
    "50+": "50+", "50plus": "50+", "50 plus": "50+",
    "65+": "65+", "65 plus": "65+"
}

df_patient["age_group"] = (
    df_patient["age_group"]
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(age_map)
)

vaccine_map = {
    "pfizer": "Pfizer",
    "pfz": "Pfizer",
    "moderna": "Moderna",
    "mdrna": "Moderna",
    "astra": "AstraZeneca",
    "astrazeneca": "AstraZeneca",
    "az": "AstraZeneca",
    "verocell": "VeroCell",
    "vero cell": "VeroCell"
}

df_vaccine["vaccine_type"] = (
    df_vaccine["vaccine_type"]
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(vaccine_map)
)





Chú thích: theo mô tả dữ liệu, file patient_info.csv được kỳ vọng có trường full_name. Tuy nhiên, trong dữ liệu thực tế nhận được, file này không chứa cột full_name. Do đó, người làm không thực hiện bước làm sạch đối với trường này.

In [ ]:
##4. Chuẩn hoá trường số & ngày tháng: dose_number → số nguyên; vaccination_date & report_date → YYYY-MM-DD.

# Chuẩn hoá dose_number → số nguyên
import pandas as pd

df_vaccine["dose_number"] = (
    df_vaccine["dose_number"]
    .astype(str)
    .str.extract(r"(\d+)")
    .astype("Int64")
)

# Chuẩn hoá vaccination_date → YYYY-MM-DD
df_vaccine["vaccination_date"] = pd.to_datetime(
    df_vaccine["vaccination_date"],
    errors="coerce",
    dayfirst=True
)
df_vaccine["vaccination_date"] = df_vaccine["vaccination_date"].dt.strftime("%Y-%m-%d")


# Chuẩn hoá report_date → YYYY-MM-DD
df_reaction["report_date"] = pd.to_datetime(
    df_reaction["report_date"],
    errors="coerce",
    dayfirst=True
)
df_reaction["report_date"] = df_reaction["report_date"].dt.strftime("%Y-%m-%d")


C:\Users\thuy linh\AppData\Local\Temp\ipykernel_20504\2338881943.py:14: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_vaccine["vaccination_date"] = pd.to_datetime(
C:\Users\thuy linh\AppData\Local\Temp\ipykernel_20504\2338881943.py:25: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_reaction["report_date"] = pd.to_datetime(


In [ ]:
##5. Xử lý dữ liệu thiếu và giá trị bất thường trong tất cả các bảng.

#Bảng patient_info
# 1. Loại bỏ bản ghi không có patient_id
df_patient = df_patient.dropna(subset=["patient_id"])

# 2. Chuẩn hoá giá trị thiếu
df_patient["gender"] = df_patient["gender"].fillna("Khong xac dinh")
df_patient["age_group"] = df_patient["age_group"].fillna("Khong xac dinh")



#Bảng reaction_log_clean
# 1.Chuẩn hoá patient_id
df_reaction["patient_id"] = (
    df_reaction["patient_id"]
    .astype(str)
    .str.strip()
    .str.upper()
)


# 2.Chuẩn hoá reaction_type
df_reaction["reaction_type"] = (
    df_reaction["reaction_type"]
    .astype(str)
    .str.strip()
    .str.lower()
)

df_reaction.loc[
    df_reaction["reaction_type"].isin(
        ["none", "không phản ứng", "khong phan ung"]
    ),
    "reaction_type"
] = "none"

df_reaction.loc[
    df_reaction["reaction_type"].isin(
        ["đau tay", "dau tay", "sốt", "sot"]
    ),
    "reaction_type"
] = "reaction"

# các giá trị còn lại (!@#, rác…) → none
df_reaction.loc[
    ~df_reaction["reaction_type"].isin(["none", "reaction"]),
    "reaction_type"
] = "none"



# 3.Chuẩn hoá severity
df_reaction["severity"] = (
    df_reaction["severity"]
    .astype(str)
    .str.strip()
    .str.lower()
)

df_reaction.loc[df_reaction["severity"] == "severe", "severity"] = "3"

df_reaction["severity"] = pd.to_numeric(
    df_reaction["severity"],
    errors="coerce"
).fillna(0).astype(int)

#4 Sửa lỗi logic giữa reaction_type & severity
df_reaction.loc[
    df_reaction["reaction_type"] == "none",
    "severity"
] = 0
#5.Chuẩn hoá report_date
df_reaction["report_date"] = pd.to_datetime(
    df_reaction["report_date"],
    errors="coerce"
)



#Bảng vaccination_record_clean

# 1. Loại bản ghi không có patient_id
df_reaction = df_reaction.dropna(subset=["patient_id"])

# 2. Chuẩn hoá phản ứng thiếu
df_reaction["reaction_type"] = df_reaction["reaction_type"].fillna("None")

# 3. Chuẩn hoá mức độ phản ứng thiếu
df_reaction["severity"] = df_reaction["severity"].fillna(0)



print("Patient NA:")
print(df_patient.isna().sum())

print("\nVaccine NA:")
print(df_vaccine.isna().sum())

print("\nReaction NA:")
print(df_reaction.isna().sum())


Patient NA:
patient_id    0
age_group     0
gender        0
dtype: int64

Vaccine NA:
patient_id          0
vaccine_type        0
dose_number         8
vaccination_date    0
dtype: int64

Reaction NA:
patient_id        0
reaction_type     0
severity          0
report_date      39
dtype: int64


In [ ]:
## Kiểm tra tính hợp lệ logic: liều tiêm trùng lặp; ngày báo cáo phản ứng trước ngày tiêm; dữ liệu không nhất quán giữa các bảng.

# Phát hiện liều tiêm trùng
duplicate_dose = df_vaccine[
    df_vaccine.duplicated(
        subset=["patient_id", "dose_number"],
        keep=False
    )
]

print("Các bản ghi liều tiêm trùng lặp:")
display(duplicate_dose)



# Chuyển về datetime để so sánh
df_vaccine["vaccination_date"] = pd.to_datetime(df_vaccine["vaccination_date"])
df_reaction["report_date"] = pd.to_datetime(df_reaction["report_date"])

# Merge tạm để kiểm tra logic ngày
reaction_check = df_reaction.merge(
    df_vaccine[["patient_id", "vaccination_date"]],
    on="patient_id",
    how="left"
)

# Phát hiện lỗi logic
invalid_date = reaction_check[
    reaction_check["report_date"] < reaction_check["vaccination_date"]
]

print("Các bản ghi phản ứng có ngày báo cáo trước ngày tiêm:")
display(invalid_date)


reaction_no_vaccine = df_reaction[
    ~df_reaction["patient_id"].isin(df_vaccine["patient_id"])
]

print("Bệnh nhân có phản ứng nhưng không có dữ liệu tiêm:")
display(reaction_no_vaccine)


vaccine_no_patient = df_vaccine[
    ~df_vaccine["patient_id"].isin(df_patient["patient_id"])
]

print("Bản ghi tiêm không có thông tin bệnh nhân:")
display(vaccine_no_patient)


patient_no_vaccine = df_patient[
    ~df_patient["patient_id"].isin(df_vaccine["patient_id"])
]

print("Bệnh nhân chưa có dữ liệu tiêm:")
display(patient_no_vaccine)


print(f"Số liều tiêm trùng lặp: {duplicate_dose.shape[0]}")
print(f"Số phản ứng có ngày sai logic: {invalid_date.shape[0]}")
print(f"Số phản ứng không có tiêm: {reaction_no_vaccine.shape[0]}")
print(f"Số tiêm không có bệnh nhân: {vaccine_no_patient.shape[0]}")


Các bản ghi liều tiêm trùng lặp:


,patient_id,vaccine_type,dose_number,vaccination_date
0,BN535,dpt,1,2024-02-23
1,BN520,covid19,3,2024-03-07
9,BN500,mmr,1,2024-04-04
11,BN522,mmr,1,2024-03-18
20,BN535,d.p.t,1,2024-04-19
21,BN503,covid,3,2024-04-20
25,BN512,dpt,1,2024-05-26
28,BN506,mmr,1,2024-04-20
31,BN503,covid,3,2024-05-13
40,BN501,d.p.t,2,2024-03-25


Các bản ghi phản ứng có ngày báo cáo trước ngày tiêm:


,patient_id,reaction_type,severity,report_date,vaccination_date
1,BN505,Đau tay,severe,2024-03-29,2024-05-11
2,BN505,Đau tay,severe,2024-03-29,2024-04-19
18,BN517,None,2,2024-02-13,2024-05-10
22,BN525,sốt,1,2024-01-03,2024-05-25
27,BN528,Sốt,0,2024-01-13,2024-01-22
34,BN512,!@#,1,2024-02-26,2024-05-26
36,BN512,!@#,1,2024-02-26,2024-03-25
46,BN520,sốt,0,2024-04-12,2024-05-14
54,BN513,đau tay,0,2024-05-07,2024-05-12
55,BN506,!@#,2,2024-03-05,2024-04-20


Bệnh nhân có phản ứng nhưng không có dữ liệu tiêm:


...

Bản ghi tiêm không có thông tin bệnh nhân:


,patient_id,vaccine_type,dose_number,vaccination_date


Bệnh nhân chưa có dữ liệu tiêm:


,patient_id,age_group,gender
7,BN507,18-49,M
11,BN511,0-5,M
18,BN518,6-17,M
32,BN532,50+,M
34,BN534,50+,M
36,BN536,0-5,M
39,BN539,18-49,F


Số liều tiêm trùng lặp: 20
Số phản ứng có ngày sai logic: 32
Số phản ứng không có tiêm: 11
Số tiêm không có bệnh nhân: 0


In [19]:
#- Đồng bộ patient_id để đảm bảo khả năng merge ở các nhiệm vụ sau.
id_patient = set(df_patient["patient_id"])
id_vaccine = set(df_vaccine["patient_id"])
id_reaction = set(df_reaction["patient_id"])

print("ID chung patient & vaccine:", len(id_patient & id_vaccine))
print("ID chung patient & reaction:", len(id_patient & id_reaction))
print("ID chung vaccine & reaction:", len(id_vaccine & id_reaction))



only_patient = id_patient - id_vaccine - id_reaction
only_vaccine = id_vaccine - id_patient
only_reaction = id_reaction - id_patient

print("Chỉ có trong patient_info:", only_patient)
print("Chỉ có trong vaccination_record:", only_vaccine)
print("Chỉ có trong reaction_log:", only_reaction)



test_merge = df_patient.merge(
    df_vaccine,
    on="patient_id",
    how="inner"
)

print("Số dòng sau merge thử:", test_merge.shape[0])


ID chung patient & vaccine: 33
ID chung patient & reaction: 36
ID chung vaccine & reaction: 29
Chỉ có trong patient_info: set()
Chỉ có trong vaccination_record: set()
Chỉ có trong reaction_log: set()
Số dòng sau merge thử: 59


In [21]:
df_patient.to_csv(
    "../data_clean/patient_info_clean.csv",
    index=False,
    encoding="utf-8"
)

df_vaccine.to_csv(
    "../data_clean/vaccination_record_clean.csv",
    index=False,
    encoding="utf-8"
)

df_reaction.to_csv(
    "../data_clean/reaction_log_clean.csv",
    index=False,
    encoding="utf-8"
)
